In [5]:
import bubble_contour_analysis as bca
import cv2

input = "スクショ.png"
graying = cv2.imread(input, cv2.IMREAD_GRAYSCALE)
binary = bca.preprocess_frame(graying)
conditions = {"min_contour_area": 5000, "max_contour_area": 20000}
filtered_contours = detect_and_filter_contours(binary, conditions)
for contour in filtered_contours:
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        reference_point = find_reference_point(contour, (cx, cy))
        angles = calculate_angle_from_reference(contour, (cx, cy), reference_point)
        
        draw_and_save_angle_images(frame, [contour], [angles], directories["angles"])
        draw_contours_and_centroid(frame, [contour], directories["contours"])
# オリジナル、グレースケール、二値化画像の保存
cv2.imwrite(os.path.join(directories["original"], f"frame_{frame_count}.bmp"), frame)
cv2.imwrite(os.path.join(directories["gray"], f"frame_{frame_count}.bmp"), cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
cv2.imwrite(os.path.join(directories["binary"], f"frame_{frame_count}.bmp"), binary)


error: OpenCV(4.8.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<1, -1, -1>, cv::impl::(anonymous namespace)::Set<0, 2, 5>, cv::impl::(anonymous namespace)::NONE>::CvtHelper(cv::InputArray, cv::OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<1, -1, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 2, 5>, sizePolicy = cv::impl::(anonymous namespace)::NONE]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [62]:
#coding : UTF-8
import cv2
import numpy as np


input = "スクショ.png"
mid = "gray.png"
output = "edge.png"

graying = cv2.imread(input, cv2.IMREAD_GRAYSCALE)
filtered = cv2.GaussianBlur(graying, (11, 11), 0)
kernel = np.ones((3, 3),np.uint8)
opened = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel, iterations=2)
detedge = cv2.Canny(opened, 14, 16)

cv2.imwrite(output, detedge)

True

In [13]:
import os
import cv2
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcParams["font.size"] = 20
import pandas as pd

file_path = 'スクショ.png'
basename = os.path.splitext(os.path.basename(file_path))[0]


# In[2]:


### 画像読み込み
img = cv2.imread(file_path, 1)
# 画像の高さと幅を取得
h, w, c = img.shape
# 拡大(拡大することで輪郭がぼやける。このぼやけにより境界を識別しやすくする)
scale = 3
img_resize = cv2.resize(img, (w * scale, h * scale))

### 画像処理
# グレースケールに変換
img_gray = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
# ガウシアンによるスムージング処理（ぼかし）
img_blur = cv2.GaussianBlur(blurred_image, (5,5), 0)
img_equa = cv2.equalizeHist(img_gray)



# 二値化と大津処理
r, dst = cv2.threshold(img_equa, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# モルフォロジー膨張処理
kernel = np.ones((3,3), np.uint8)
dst = cv2.dilate(dst, kernel, iterations = 1)

# 画像ファイルに保存
cv2.imwrite(basename + '_thresholds.jpg', dst)


# In[3]:


# もし画像欠けがあった場合に塗りつぶす処理
dst_fill = ndimage.binary_fill_holes(dst).astype(int) * 255
cv2.imwrite(basename + '_thresholds_fill.jpg', dst_fill)


# In[4]:


# 境界を検出して描画する
contours, _ = cv2.findContours(dst_fill.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
img_contour = cv2.drawContours(img_resize, contours, -1, (0,0,255), 1)
cv2.imwrite(basename + '_counter.jpg', img_contour)


# In[5]:

# フィルタリングされた輪郭を格納するリスト

Areas = []
with open(basename + '_data.csv', 'w') as f:
    for i, contour in enumerate(contours):
        # 面積
        area = cv2.contourArea(contour)
        area = area / 1000
        Areas.append(area)
        # 輪郭の重心
        M = cv2.moments(contour)
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])  

        # 輪郭（境界）の長さ
        perimeter = cv2.arcLength(contours[i], True)        
        
        # 画像に出力する
        img2 = cv2.drawContours(img_resize, contours, i, (0, 0, 255), 3)
        cv2.putText(img2, str('{:.1f}'.format(area)), (cx, cy),
                    cv2.FONT_HERSHEY_PLAIN,  # フォントタイプ
                    3,  # 文字サイズ
                    (0, 0, 0),  # 文字色：白(255, 255, 255)　黒(0, 0, 0)
                    2,  # 文字太さ
                    cv2.LINE_AA)

        if i == (len(contours)-1):
            img2_resize = cv2.resize(img2, (w, h))
            cv2.imwrite(basename + '_' + str(i) + '.jpg', img2_resize)

        # csvファイルに保存
        if i == 0:
            my_columns_list = ['ID', 'Area', 'x_center_of_gravity', 'y_center_of_gravity', 'Perimeter']
            my_columns_str = ','.join(my_columns_list)
            f.write(my_columns_str + '\n')
        else:
            my_data_list = [str(i), str(area), str(cx), str(cy), str(perimeter)]
            my_data_str = ','.join(my_data_list)
            f.write(my_data_str + '\n')
    
Area_sum = sum(Areas)
print('Area_sum', Area_sum)



# In[6]:


# ヒストグラム表示
fig = plt.figure(figsize=(8,6))
plt.title("histogram")
plt.xlabel('Area')
plt.ylabel('frequency')
plt.tick_params()
plt.grid()
plt.hist(Areas, bins=10, rwidth=0.9) # binsは区分数
plt.savefig(basename + '_histogram.jpg')
plt.close()


# In[7]:


# 面積を割合で出力する
df = pd.read_csv(basename + '_data.csv')
df[r'Area[%]'] = df['Area'] / Area_sum * 100
df.to_csv(basename + '_data_2.csv')
df


# In[8]:


### 累積分布図を表示する
# ヒストグラムデータを抽出
values, base = np.histogram(df['Area'], bins=40) # binsは区分数

# 要素を足し合わせて、numpyアレイで出力する
y = np.cumsum(values)

# グラフをプロット
fig = plt.figure(figsize=(8,6))
plt.plot(base[:-1], y, color='black', linewidth = 4)

# 以下、グラフオプション
plt.xlabel('Area')
plt.ylabel('p')
#plt.ylim(0, 1)
# 目盛り表記を強制的に整数にする
plt.gca().get_yaxis().set_major_locator(ticker.MaxNLocator(integer=True))
plt.grid()
plt.title('Cumulative distribution')
plt.savefig(basename + '_cumulative_distribution.jpg')
plt.close()

Area_sum 13903.315000000015
